In [1]:
from transformers import pipeline
from util.summary_generator import SummaryGenerator
from util.memory import Memory
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

C:\Users\hleie\anaconda3\envs\quang2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Dialogue Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-3b", device_map="auto",
                                             torch_dtype=torch.bfloat16, cache_dir='dolly-v2-3b')

## Load Summary Model

In [3]:
summary_pipeline = pipeline('summarization', model='kabita-choudhary/finetuned-bart-for-conversation-summary')
summary_generator = SummaryGenerator(summary_pipeline)

## Load sentence embedding model

In [4]:
sentence_model = SentenceTransformer(
    'sentence-transformers/paraphrase-MiniLM-L6-v2',
    cache_folder='paraphrase-MiniLM-L6-v2'
)

## Define dialogue

In [5]:
dialogue = """Laurie: So, what are your plans for this weekend?\n
Christie: I don’t know. Do you want to get together or something?\n
Sarah: How about going to see a movie? Cinemax 26 on Carson Boulevard is showing Enchanted. Laurie: That sounds like a good idea. Maybe we should go out to eat beforehand.\n
Sarah: It is fine with me. Where do you want to meet?\n
Christie: Let’s meet at Summer Pizza House. I have not gone there for a long time.\n
Laurie: Good idea again. I heard they just came up with a new pizza. It should be good because Summer Pizza House always has the best pizza in town.\n
Sarah: When should we meet?\n
Christie: Well, the movie is shown at 2:00PM, 4:00PM, 6:00PM and 8:00PM.\n
Laurie: Why don’t we go to the 2:00PM show? We can meet at Summer Pizza House at noon. That will give us plenty of time to enjoy our pizza.\n
Sarah: My cousin Karen is in town. Can I bring her along? I hate to leave her home alone.\n
Christie: Karen is in town? Yes, bring her along. Laurie, you remember Karen? We met her at Sara’s high school graduation party two years ago.\n
Laurie: I do not quite remember her. What does she look like?\n
Sarah: She has blond hair, she is kind of slender, and she is about your height.\n
Laurie: She wears eyeglasses, right?\n
Sarah: Yes, and she was playing the piano off and on during the party.\n
Laurie: I remember her now. Yes, do bring her along Sara. She is such a nice person, and funny too.\n
Sarah: She will be happy to meet both of you again.\n
Christie: What is she doing these days?\n
Sarah: She graduated last June, and she will start her teaching career next week when the new school term begins.\n
Laurie: What grade is she going to teach?\n
Sarah: She will teach kindergarten. She loves working with kids, and she always has such a good rapport with them\n
Christie: Kindergarten? She must be a very patient person. I always think kindergarten is the most difficult class to teach. Most of the kids have never been to school, and they have e never been away from mommy for long.\n
Sarah:  I think Karen will do fine. She knows how to handle young children\n
Laurie: I think the first few weeks will be tough. However, once the routine is set, it should not be too difficult to teach kindergarten.\n
Christie: You are right. The kids might even look forward to going to school since they have so many friends to play with.\n
Sarah: There are so many new things for them to do at school too. They do a lot of crafts in kindergarten. I am always amazed by the things kindergarten teachers do.\n
Laurie: Yes, I have seen my niece come home with so many neat stuff.\n
Christie: Maybe we can ask Karen to show us some of the things that we can do for this Halloween.\n
Laurie: Maybe we can stop by the craft store after the movie. What do you think, Sara?\n
Sarah: I will talk to her. I think she will like that. It will help her with school projects when Halloween comes.\n
Christie: Michael’s is a good store for crafts. It always carries a variety of things, and you can find almost anything there.\n
Laurie: There is a Michaels store not far away from Cinemax 26. I believe it is just around the corner, on Pioneer Avenue. We can even walk over there.\n
Sarah: So, we plan to meet for pizza at noon, go to the movies at two, and shop at Michael’s afterward. Right?\n
Laurie and Christie: Yes."""

## Generate Memories (with sampling)

In [6]:
sentence_list = summary_generator.generate_memories(dialogue, max_length=128, min_length=30, do_sample=True)
print(f"Memories: {sentence_list}")

C:\Users\hleie\anaconda3\envs\quang2\lib\site-packages\transformers\generation\utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Memories: ['Christie, Laurie, Sarah and Sarah are going to see "Enchanted" at Cinemax 26 on Carson Boulevard at 2 PM.', 'They will meet at Summer Pizza House at noon.', "After the movie, they will go to Michael's for pizza and meet Sarah's cousin Karen, who will start her new job as a kindergarten teacher next week.", '.']


## Generate Memories (without sampling)

In [7]:
sentence_list = summary_generator.generate_memories(dialogue, max_length=128, min_length=30, do_sample=False)
print(f"Memories: {sentence_list}")

Memories: ['Christie, Laurie, Sarah and Sarah are going to see "Enchanted" at Cinemax 26 on Carson Boulevard at 2 PM.', "They will meet at Summer Pizza House at noon and go to Michael's afterwards.", "They are also going to meet Karen, Sarah's cousin, who graduated last June and will start her teaching career next week as a kindergarten teacher.", '.']


## Get sentence rating for each sentence

In [8]:
sentence_rating_list = [1 for s in sentence_list]

## Add sentences and sentence ratings to Memory

In [9]:
memory = Memory(list(), list())
memory.add_sentences(sentence_list, sentence_rating_list)
print(f'Key Sentences: {memory.key_sentences}')

Key Sentences: ['Christie, Laurie, Sarah and Sarah are going to see "Enchanted" at Cinemax 26 on Carson Boulevard at 2 PM.', "They will meet at Summer Pizza House at noon and go to Michael's afterwards.", "They are also going to meet Karen, Sarah's cousin, who graduated last June and will start her teaching career next week as a kindergarten teacher.", '.']


In [11]:
# Number of sentences used in initial prompt
num_sentences = 2

# Player and NPC Name:
player_name = 'Laurie'
npc_name = 'Sarah'

# Start conversation history
conversation = ""

# Start dialogue loop
# while True:
for i in range(3):
    
    # Get user input
    user_input = input(f'{player_name}: ')
    print(f"\n{player_name}: " + user_input)

    # Generate initial prompt based on current user input
    initial_prompt = memory.generate_prompt(user_input, sentence_model, num_sentences=num_sentences)
    print(f'\nInitial Prompt: {initial_prompt}')

    # Add user input to conversation history
    conversation += f"\n{player_name}:\n" + user_input + f"\n{npc_name}: "

    # Add initial prompt to conversation history to generate model input
    input_text = initial_prompt + "\n" + conversation
    input_length = len(input_text)

    # Tokenize input
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

    # Generate respones from dialogue model
    generation_output = model.generate(
        input_ids=input_ids, max_new_tokens=64, temperature=0.2, do_sample=True
    )

    # Decode response
    output = tokenizer.decode(generation_output[0])

    # Limit reponse to NPC response
    split_string = output[input_length:].split('\n', 2)
    response = split_string[1] 
    print(f"\n{npc_name}: " + response)

    # Add response to conversation history
    conversation += response

Laurie:  Hey, how was Karens first day at work?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Laurie: Hey, how was Karens first day at work?

Initial Prompt: They will meet at Summer Pizza House at noon and go to Michael's afterwards.They are also going to meet Karen, Sarah's cousin, who graduated last June and will start her teaching career next week as a kindergarten teacher.

Sarah: It was great! She was very excited to start and the kids were so engaged. She is going to be a great teacher!


Laurie:  What is her job again?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Laurie: What is her job again?

Initial Prompt: .They are also going to meet Karen, Sarah's cousin, who graduated last June and will start her teaching career next week as a kindergarten teacher.

Sarah: She is a kindergarten teacher.


Laurie:  Ah, I see. Where should we meet later?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Laurie: Ah, I see. Where should we meet later?

Initial Prompt: Christie, Laurie, Sarah and Sarah are going to see "Enchanted" at Cinemax 26 on Carson Boulevard at 2 PM.They will meet at Summer Pizza House at noon and go to Michael's afterwards.

Sarah: We are going to see "Enchanted" at Cinemax 26 on Carson Boulevard at 2 PM.
